In [1]:
# imports
import datasets
import evaluate
import numpy as np
from datasets import load_dataset, Dataset
from datasets import Features
from datasets import Sequence
from datasets import ClassLabel
from datasets import Value
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, BertConfig, AutoModelForSequenceClassification, \
    DataCollatorForTokenClassification, TrainingArguments, Trainer
import torch

In [2]:
from huggingface_hub import notebook_login

#notebook_login()

In [3]:
RES_PATH = os.path.abspath("../resources/data/") + "/"

In [4]:
def load_dataset_dict(dataset_name, dataset_path, limit=None):
    train_file = dataset_path + dataset_name + "/train.json"
    test_file = dataset_path + dataset_name + "/test.json"
    dev_file = dataset_path + dataset_name + "/dev.json"

    return load_dataset("json", data_files={"train":train_file, "validation":dev_file, "test":test_file},
                        features=Features({
                            "id": Value(dtype="string", id=None),
                            "label": Sequence(ClassLabel(num_classes=3, names=["B", "I", "O"]), length=-1, id=None),
                            "sentence": Sequence(Value(dtype="string", id=None), length=-1, id=None),
                        })).rename_column("label", "labels")

In [5]:
laptop_dataset_dict = load_dataset_dict("test", RES_PATH)
#laptop_dataset_dict = load_dataset_dict("laptop", RES_PATH)

In [6]:
laptop_dataset_dict["train"][0]

{'id': '0',
 'labels': [0, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2],
 'sentence': ['Keyboard',
  'is',
  'great',
  'but',
  'primary',
  'and',
  'secondary',
  'control',
  'buttons',
  'could',
  'be',
  'more',
  'durable',
  '.']}

In [7]:
#label_list = laptop_dataset_dict["train"].features["label"].feature.names # ["B", "I", "O"]
label_list = ["B", "I", "O"]
label_list

['B', 'I', 'O']

In [8]:
bert_auto_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [9]:
example = laptop_dataset_dict["train"][0]
tokenized_input = bert_auto_tokenizer(example["sentence"], is_split_into_words=True, padding=True, truncation=True)
tokens = bert_auto_tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'keyboard',
 'is',
 'great',
 'but',
 'primary',
 'and',
 'secondary',
 'control',
 'buttons',
 'could',
 'be',
 'more',
 'durable',
 '.',
 '[SEP]']

In [10]:
def tokenize_and_align_labels(batch_data):
    tokenized_inputs = bert_auto_tokenizer(batch_data["sentence"], is_split_into_words=True, padding=True, truncation=True)

    labels = []
    for i, label in enumerate(batch_data["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100 -> ignored by PT
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [11]:
tokenized_laptop_dataset_dict = laptop_dataset_dict.map(tokenize_and_align_labels, batched=True)

In [12]:
tokenized_laptop_dataset_dict["train"].features

{'id': Value(dtype='string', id=None),
 'labels': Sequence(feature=ClassLabel(names=['B', 'I', 'O'], id=None), length=-1, id=None),
 'sentence': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [13]:
data_collator = DataCollatorForTokenClassification(tokenizer=bert_auto_tokenizer, padding=True)

In [14]:
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

In [15]:
labels = [label_list[i] for i in example["labels"]]

In [16]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = []
    for prediction, label in zip(predictions, labels):
        for (p, l) in zip(prediction, label):
            if l != -100:
                true_predictions.append(label_list[p])

    true_labels = []
    for prediction, label in zip(predictions, labels):
        for (p, l) in zip(prediction, label):
            if l != -100:
                true_labels.append(label_list[l])

    true_labels = [label2id[item] for item in true_labels]
    true_predictions = [label2id[item] for item in true_predictions]

    results = {}
    results.update(accuracy_metric.compute(predictions=true_predictions, references=true_labels))
    results.update(precision_metric.compute(predictions=true_predictions, references=true_labels, average="micro"))
    results.update(recall_metric.compute(predictions=true_predictions, references=true_labels, average="micro"))
    results.update(f1_metric.compute(predictions=true_predictions, references=true_labels, average="micro"))

    return results

In [17]:
id2label = {
    0: "B",
    1: "I",
    2: "O",
}
label2id = {
    "B": 0,
    "I": 1,
    "O": 2
}

In [18]:
bert_token_classificator = AutoModelForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
training_args = TrainingArguments(
    output_dir="aspect_extraction_laptop_reviews",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    #save_steps=100.0,
    #save_total_limit=2,
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True)

In [20]:
trainer = Trainer(
    model=bert_token_classificator,
    args=training_args,
    train_dataset=tokenized_laptop_dataset_dict["train"],
    eval_dataset=tokenized_laptop_dataset_dict["test"],
    tokenizer=bert_auto_tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning https://huggingface.co/jannikseus/aspect_extraction_laptop_reviews into local empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [21]:
trainer.train()

/Users/jannik/DataspellProjects/nlp-project-ae/venv/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=2, training_loss=1.017417311668396, metrics={'train_runtime': 36.5895, 'train_samples_per_second': 0.82, 'train_steps_per_second': 0.055, 'total_flos': 283256959500.0, 'train_loss': 1.017417311668396, 'epoch': 1.0})

In [22]:
trainer.push_to_hub()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Upload file pytorch_model.bin:   0%|          | 1.00/253M [00:00<?, ?B/s]

To https://huggingface.co/jannikseus/aspect_extraction_laptop_reviews
   2f615ce..169b9f9  main -> main



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
trainer.evaluate()

{'eval_loss': 0.8763446807861328,
 'eval_accuracy': 0.8134328358208955,
 'eval_precision': 0.8134328358208955,
 'eval_recall': 0.8134328358208955,
 'eval_f1': 0.8134328358208955,
 'eval_runtime': 0.6853,
 'eval_samples_per_second': 11.674,
 'eval_steps_per_second': 1.459,
 'epoch': 1.0}

In [24]:
trainer.predict(tokenized_laptop_dataset_dict["validation"])

PredictionOutput(predictions=array([[[-1.59245759e-01, -1.41889662e-01,  1.41037703e-02],
        [-1.87129647e-01, -1.41412169e-01,  3.88076067e-01],
        [-2.60223389e-01, -2.33298510e-01,  6.85155749e-01],
        [-3.32794726e-01, -9.41025987e-02,  4.24694598e-01],
        [-1.52201429e-01, -4.07698154e-01,  2.64618754e-01],
        [-1.82764560e-01, -3.50165427e-01,  1.98252276e-01],
        [-3.36912513e-01, -1.74226105e-01,  2.43389666e-01],
        [-2.74318337e-01, -2.56763607e-01,  5.15795827e-01],
        [-2.51258790e-01, -3.94350529e-01,  2.15758026e-01],
        [-1.68532491e-01, -3.06511074e-01,  3.58344793e-01],
        [-1.97740957e-01, -7.90176839e-02,  5.69956601e-01],
        [-1.47146851e-01, -1.66778803e-01,  6.04994416e-01],
        [-2.67290771e-01, -2.36941904e-01,  3.11656833e-01],
        [-4.98436183e-01, -1.33061662e-01,  3.92339081e-01],
        [-2.05842763e-01, -2.47040123e-01, -1.43875405e-02],
        [ 2.94050723e-02, -2.51022339e-01,  5.15675068e-